In [1]:
""" Inspired by example from
https://github.com/Vict0rSch/deep_learning/tree/master/keras/recurrent
Uses the TensorFlow backend
The basic idea is to detect anomalies in a time-series.
"""
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from numpy import arange, sin, pi, random
import math
np.random.seed(1234)

# Global hyper-parameters
sequence_length = 100
random_data_dup = 5  # each sample randomly duplicated between 0 and 9 times, see dropin function
epochs = 1
batch_size = 50


Using TensorFlow backend.


In [2]:
v1 = pd.read_csv('test_v1.csv')

In [3]:
v1.drop('Time', axis=1, inplace=True)
v1.drop('Date', axis=1, inplace=True)

In [4]:
v1.sort_index(ascending=False, inplace=True)

In [5]:
v1 = v1.reset_index()

In [6]:
v1.drop('index',axis=1, inplace=True)

In [7]:
v1 = v1.interpolate()
#v1.fillna(method='pad') # https://pandas.pydata.org/pandas-docs/stable/missing_data.html

In [8]:
v1.shape

(786422, 8)

In [9]:
v1k = v1[:1000]
active_power = v1k.Global_active_power
reactive_power = v1k.Global_reactive_power
voltage = v1k.Voltage
intensity = v1k.Global_intensity
metering1 = v1k.Sub_metering_1
metering2 = v1k.Sub_metering_2
metering3 = v1k.Sub_metering_3

In [10]:
# v1.shape (786422, 8)
active_power = v1.Global_active_power
reactive_power = v1.Global_reactive_power
voltage = v1.Voltage
intensity = v1.Global_intensity
metering1 = v1.Sub_metering_1
metering2 = v1.Sub_metering_2
metering3 = v1.Sub_metering_3
serieses = [active_power, reactive_power, voltage, intensity, metering1, metering2, metering3]

In [11]:
# v1.shape (786422, 8)
def trim_5(series):
    while series.shape[0] % 5 != 0:
        series = series.iloc[:-1]
    return series
def avg_5(series):
    name = series.name# + 'div5'
    trimmed = trim_5(series)
    shaped = trimmed.values.reshape(int(trimmed.shape[0]/5),5)
    meaned = shaped.mean(axis=1)
    return pd.Series(meaned, name=name)
def avg_5_list(serieses):
    serieses_avg_5 = []
    for eachSeries in serieses:
        serieses_avg_5.append(avg_5(eachSeries))
    return serieses_avg_5
def first_n(serieseses, n):
    result = []
    for eachSerieses in serieseses:
        result.append(eachSerieses[:n])
    return result

In [12]:
serieses_avg_5 = avg_5_list(serieses)
serieses_avg_25 = avg_5_list(serieses_avg_5)
serieses_avg_125 = avg_5_list(serieses_avg_25)
serieses_avg_625 = avg_5_list(serieses_avg_125)
#serieses_avg_3125 = avg_5_list(serieses_avg_625)
serieseses = [serieses, serieses_avg_5, serieses_avg_25, serieses_avg_125, serieses_avg_625]

In [13]:
ssss_1250 = first_n(serieseses, 1250)

In [14]:
def plot250(serieses, scale_string, offset=0):
    plt.figure(1, figsize=(20,20))
    seriesCount = len(serieses)
    plotCount = seriesCount * 100 + 10
    for i, series in enumerate(serieses):
        toPlot = series[250*offset:250]
        plotPosition = plotCount + i + 1
        plt.subplot(plotPosition)
        plt.title(series.name + " Signal at scale: " + str(scale_string))
        plt.plot(series, 'b', alpha=0.6)
    plt.savefig("plot250_" + str(scale_string) + "_offset_" + str(offset), dpi=100)

In [15]:
def dropin(X, y):
    """ The name suggests the inverse of dropout, i.e. adding more samples. See Data Augmentation section at
    http://simaaron.github.io/Estimating-rainfall-from-weather-radar-readings-using-recurrent-neural-networks/
    :param X: Each row is a training sequence
    :param y: Tne target we train and will later predict
    :return: new augmented X, y
    """
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    X_hat = []
    y_hat = []
    for i in range(0, len(X)):
        for j in range(0, np.random.randint(0, random_data_dup)):
            X_hat.append(X[i, :])
            y_hat.append(y[i])
    return np.asarray(X_hat), np.asarray(y_hat)



In [16]:
def z_norm_multivariate(series):
    series -= series.mean()
    series /= series.std()
    return series

In [17]:

def z_norm(result):
    result_mean = result.mean()
    result_std = result.std()
    result -= result_mean
    result /= result_std
    return result, result_mean

In [18]:

def get_split_prep_data(data, train_start, train_end, test_start, test_end):
    print("Length of Data", len(data))

    # train data
    print ("Creating train data...")

    result = []
    for index in range(train_start, train_end - sequence_length):
        result.append(data[index: index + sequence_length])
    result = np.array(result)  # shape (samples, sequence_length)
    result, result_mean = z_norm(result)

    print ("Mean of train data : ", result_mean)
    print ("Train data shape  : ", result.shape)

    train = result[train_start:train_end, :]
    np.random.shuffle(train)  # shuffles in-place
    X_train = train[:, :-1]
    y_train = train[:, -1]
    X_train, y_train = dropin(X_train, y_train)

    # test data
    print ("Creating test data...")

    result = []
    for index in range(test_start, test_end - sequence_length):
        result.append(data[index: index + sequence_length])
    result = np.array(result)  # shape (samples, sequence_length)
    result, result_mean = z_norm(result)
    
    X_test = result[:, :-1]
    y_test = result[:, -1]

    print("Shape X_train", np.shape(X_train))
    print("Shape y_train", np.shape(y_train))
    print("Shape X_test", np.shape(X_test))
    print("Shape y_test", np.shape(y_test))

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    return X_train, y_train, X_test, y_test



In [19]:
def build_model():
    model = Sequential()
    layers = {'input': 1, 'hidden1': 64, 'hidden2': 256, 'hidden3': 100, 'output': 1}

    model.add(LSTM(
            input_shape=(sequence_length - 1, layers['input']),
            units=layers['hidden1'],
            return_sequences=True,
            #stateful=True,
            #batch_input_shape = (None,sequence_length-1,1)
    ))
    model.add(Dropout(0.2))

    model.add(LSTM(
            layers['hidden2'],
            return_sequences=True,
            #stateful=True
    ))
    model.add(Dropout(0.2))

    model.add(LSTM(
            layers['hidden3'],
            return_sequences=False,
            #stateful=True
    ))
    model.add(Dropout(0.2))

    model.add(Dense(
            units=layers['output']))
    model.add(Activation("linear"))


    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    print ("Compilation Time : ", time.time() - start)
    return model



In [20]:
def save_csv(signal, predicted, mse, title, view_range):
    df = pd.DataFrame({title:signal, "predicted": predicted, "mse":mse})    
    df.to_csv(title + '_predicted_mse_'+ str(view_range) + '.csv')

In [21]:
#run network
def run(series, view_range, scale_string=""):
    model = build_model()
    data = series.values
    title = series.name + " at scale: " + scale_string
    global_start_time = time.time()
    X_train, y_train, X_test, y_test = get_split_prep_data(series.values, 0, view_range, 0, view_range)
    try:
        print("Training...")
        model.fit(
                X_train, y_train,
                batch_size=batch_size, 
            epochs=epochs, 
            validation_split=0.05,
            shuffle=False
        )
        print("Predicting...")
        predicted = model.predict(X_test)
        print("Reshaping predicted")
        predicted = np.reshape(predicted, (predicted.size,))
    except KeyboardInterrupt:
        print("prediction exception")
        print ('Training duration (s) : ', time.time() - global_start_time)

    try:
        actual_plot = y_test[:len(y_test)]
        predicted_plot = predicted[:len(y_test)]
        mse = ((y_test - predicted) ** 2)
        triplePlot()
        mpl.rcParams.update({'font.size': 8})
#         plt.clf()
#         plt.figure(1, figsize=(30,15))
#         plt.subplot(311)
#         plt.title("Actual Signal With Anomalies: " + title)
#         plt.plot(, 'b', alpha=0.6)
#         plt.subplot(312)
#         plt.title("Predicted Signal")
#         plt.plot(predicted[:len(y_test)], 'g', alpha=0.6)
#         plt.subplot(313)
#         plt.title("Squared Error")
        #plt.plot(mse, 'r', alpha=0.6)
        #plt.tight_layout()
        #plt.savefig(title + '_lstm_' + '.png' , bbox_inches='tight', dpi=200)
        save_csv(y_test[:len(y_test)],predicted[:len(y_test)], mse, title, view_range)
    except Exception as e:
        print("plotting exception")
        print (str(e))
    print ('Training duration (s) : ', time.time() - global_start_time)


In [25]:
'''
active_power = v1k.Global_active_power
reactive_power = v1k.Global_reactive_power
voltage = v1k.Voltage
intensity = v1k.Global_intensity
metering1 = v1k.Sub_metering_1
metering2 = v1k.Sub_metering_2
metering3 = v1k.Sub_metering_3
'''
#[serieses, serieses_avg_5, serieses_avg_25, serieses_avg_125, serieses_avg_625]
for eachSeries in serieses_avg_625:
    run(eachSeries, 1000, "5^" + "4")

Compilation Time :  0.05466413497924805
Length of Data 1258
Creating train data...
Mean of train data :  1.3627338794
Train data shape  :  (900, 100)
X shape: (900, 99)
y shape: (900,)
Creating test data...
Shape X_train (1784, 99)
Shape y_train (1784,)
Shape X_test (900, 99)
Shape y_test (900,)
Training...
Train on 1694 samples, validate on 90 samples
Epoch 1/1
1694/1694 [==============================] - 106s - loss: 0.7792 - val_loss: 0.7461
Predicting...
Reshaping predicted
Training duration (s) :  137.79941201210022
Compilation Time :  0.060101985931396484
Length of Data 1258
Creating train data...
Mean of train data :  0.117135573445
Train data shape  :  (900, 100)
X shape: (900, 99)
y shape: (900,)
Creating test data...
Shape X_train (1854, 99)
Shape y_train (1854,)
Shape X_test (900, 99)
Shape y_test (900,)
Training...
Train on 1761 samples, validate on 93 samples
Epoch 1/1
1761/1761 [==============================] - 115s - loss: 0.8320 - val_loss: 0.7230
Predicting...
Reshapi

In [35]:
# v1.shape (786422, 8)
scale = 0
for i, eachSerieses in enumerate(serieseses):
    scale = i
    for eachSeries in eachSerieses:
        run(eachSeries, 1000, "5^" + str(scale))


Compilation Time :  0.10749101638793945
Length of Data 786422
Creating train data...
Mean of train data :  2.56858015278
Train data shape  :  (900, 100)
X shape: (900, 99)
y shape: (900,)
Creating test data...
Shape X_train (1823, 99)
Shape y_train (1823,)
Shape X_test (900, 99)
Shape y_test (900,)
Training...
Train on 1731 samples, validate on 92 samples
Epoch 1/1
1731/1731 [==============================] - 224s - loss: 0.6811 - val_loss: 1.0986
Predicting...
Reshaping predicted
Training duration (s) :  280.21554136276245
Compilation Time :  0.13102197647094727
Length of Data 786422
Creating train data...
Mean of train data :  0.129359444444
Train data shape  :  (900, 100)
X shape: (900, 99)
y shape: (900,)
Creating test data...
Shape X_train (1738, 99)
Shape y_train (1738,)
Shape X_test (900, 99)
Shape y_test (900,)
Training...
Train on 1651 samples, validate on 87 samples
Epoch 1/1
1651/1651 [==============================] - 206s - loss: 0.2608 - val_loss: 0.1732
Predicting...
Res

KeyboardInterrupt: 